In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from skfuzzy import defuzzify

A service centre keeps spare parts and repairs
parts. 
A customer brings a failed item and receives a spare of the same type spare of the same type.

Failed parts are repaired by servers, placed on the shelf and thus become spares the shelf, and thus become spares.

The objective here is to advise a manager of the service centre on certain decision policies to keep service centre on certain decision policies to keep te customers satisfied.

Advise on the initial number of spares to keep Advise on the initial number of spares to keep delay reasonable

[source](http://ce.sharif.edu/courses/92-93/1/ce957-1/resources/root/Lectures/Lecture6&7.pdf)

The fuzzy variables are defines with their sets as:
![table.png](../images/07/variables_table.png)

The expected values are as shown below:

**Mean Delay**

![delay.png](../images/07/delay_membership.png)

**Number of Servers**

![servers.png](../images/07/servers_membership.png)

**Repair Utilization factor**

![repair.png](../images/07/util_membership.png)

**Number of Spares**

![spares.png](../images/07/spares_membership.png)

In [ ]:
class FuzzySys():
    def __init__(self):
        self.init_fuzzy()
        self.create_sys()

    def init_fuzzy(self):
        # Antecedent: Input
        # Consequent: Output
        universe = np.arange(0,1,0.05)
        # New Antecedent/Consequent objects hold universe variables and membership
        # functions
        delay = ctrl.Antecedent(universe, 'delay')
        servers = ctrl.Antecedent(universe, 'servers')
        repairutil = ctrl.Antecedent(universe, 'repairutil')
        spares = ctrl.Consequent(universe, 'spares')


        # Assign membership functions
        # Mean Delay Membership Functions
        delay['VS'] = fuzz.zmf(universe, 0.1, 0.3)
        delay['S'] = fuzz.trimf(universe, [0.1,0.3,0.5])
        delay['M'] = fuzz.smf(universe, 0.4, 0.6)

        # Servers membership functions
        servers['S'] = fuzz.zmf(universe, 0.15, 0.35)
        servers['M'] = fuzz.trimf(universe, [0.3,0.5,0.7])
        servers['L'] = fuzz.smf(universe, 0.6, 0.8)

        # repairutil
        repairutil['L'] = fuzz.zmf(universe, 0.4, 0.6)
        repairutil['M'] = fuzz.trimf(universe, [0.4,0.6,0.8])
        repairutil['H'] = fuzz.smf(universe, 0.6, 0.8)
        
        # Membership function for spares
        spares['VS'] = fuzz.zmf(universe, 0.1, 0.3)
        spares['S'] = fuzz.trimf(universe, [0.0,0.2,0.4])
        spares['RS'] = fuzz.trimf(universe, [0.25,0.35,0.45])
        spares['M'] = fuzz.trimf(universe, [0.3,0.5,0.7])
        spares['RL'] = fuzz.trimf(universe, [0.55,0.65,0.75])
        spares['L'] = fuzz.trimf(universe, [0.6,0.8,1.0])
        spares['VL'] = fuzz.smf(universe, 0.7, 0.9)
        
        
        self.delay = delay
        self.servers = servers
        self.repairutil = repairutil
        self.spares = spares

    def visualize(self, var_name):
        # Visualize the member functions using the *view()* method
        if var_name == 'delay':   self.delay.view()
        elif var_name == 'servers':    self.servers.view()
        elif var_name == 'repairutil':       self.repairutil.view()
        elif var_name == 'spares':       self.spares.view()

    def setup_rules(self):
        # Fuzzy Rules
        rule1 = ctrl.Rule(self.repairutil['L'], self.spares['S'])
        rule2 = ctrl.Rule(self.repairutil['M'], self.spares['M'])
        rule3 = ctrl.Rule(self.repairutil['H'], self.spares['L'])

        rule4 = ctrl.Rule(self.delay['VS'] & self.servers['S'], self.spares['VL'])
        rule5 = ctrl.Rule(self.delay['S'] & self.servers['S'], self.spares['L'])
        rule6 = ctrl.Rule(self.delay['M'] & self.servers['S'], self.spares['M'])
        rule7 = ctrl.Rule(self.delay['VS'] & self.servers['M'], self.spares['RL'])
        rule8 = ctrl.Rule(self.delay['S'] & self.servers['M'], self.spares['RS'])
        rule9 = ctrl.Rule(self.delay['M'] & self.servers['M'], self.spares['S'])
        rule10 = ctrl.Rule(self.delay['VS'] & self.servers['L'], self.spares['M'])
        rule11 = ctrl.Rule(self.delay['S'] & self.servers['L'], self.spares['S'])
        rule12 = ctrl.Rule(self.delay['M'] & self.servers['L'], self.spares['VS'])
        
        return [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, rule11, rule12]
    
    def create_sys(self):
        # Create Control System (for inferencing)
        self.center_ctrl = ctrl.ControlSystem(self.setup_rules())

        # Create simulation
        self.center_sim = ctrl.ControlSystemSimulation(self.center_ctrl)


    def inference(self, delay, servers, repairutil):
        # Provide inputs to the control simulation 
        self.center_sim.input['delay'] = delay
        self.center_sim.input['servers'] = servers
        self.center_sim.input['repairutil'] = repairutil
        
        # perform computation
        self.center_sim.compute()
        return self.center_sim.output['spares']

The rules defined correspond to the following set of 12 rules:

![rules_1.png](../images/07/rules_1.png)

![rules_2.png](../images/07/rules_2.png)

In [ ]:
# Creating the fuzzy system
fuzzsys = FuzzySys()

### Defined Membership Functions vs Required

**Mean Delay**

![delay.png](../images/07/delay_membership.png)

In [ ]:
fuzzsys.visualize('delay')

**Servers**

![servers.png](../images/07/servers_membership.png)

In [ ]:
fuzzsys.visualize('servers')

**Repair Utilization factor**

![repair.png](../images/07/util_membership.png)

In [ ]:
fuzzsys.visualize('repairutil')

**Number of Spares**

![spares.png](../images/07/spares_membership.png)

In [ ]:
fuzzsys.visualize('spares')

## Inferencing

In [ ]:
fuzzsys.inference(0.9,0.8,0.2)

In [ ]:
Also refer to the documentation [here](https://pythonhosted.org/scikit-fuzzy/) and the [tipping problem](https://pythonhosted.org/scikit-fuzzy/auto_examples/plot_tipping_problem.html)